# Image Localization Pipeline

This notebook processes images stored in MongoDB and performs object localization on them using a pre-trained model. The localizations are then stored back in the database and linked to their parent images.

## Setup
Import required dependencies and initialize configuration managers and pipeline components.

In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from tqdm.notebook import tqdm

from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.data_pipeline.mongodb_manager import MongoDBManager
from iris.data_pipeline.image_store_manager import ImageStoreManager
from iris.config.localization_pipeline_config_manager import LocalizationPipelineConfigManager
from iris.localization_pipeline.localizer import Localizer
from iris.models.image import Image
from iris.models.localization import Localization
from iris.utils.log import logger

## Initialize Pipeline Components

1. Initialize the data pipeline configuration manager
2. Set up MongoDB connection
3. Configure image store for accessing image data
4. Initialize the object localization model

In [ ]:
# Initialize configuration managers
config_manager = DataPipelineConfigManager()

# Initialize MongoDB manager
mongodb_config = config_manager.mongodb_config
mongodb_manager = MongoDBManager(mongodb_config)

# Initialize image store manager
image_store_config = config_manager.image_store_config
image_store_manager = ImageStoreManager(image_store_config)

# Initialize localizer
localization_config = LocalizationPipelineConfigManager()
localizer = Localizer(localization_config.model_config)

## Process Images

For each image in the database:
1. Run object localization to detect objects
2. Create Localization objects for each detected object
3. Store new localizations in MongoDB
4. Update the parent image with references to the new localizations
5. Clean up old localizations

In [ ]:
with mongodb_manager as db:
    images: list[Image] = db.find_all(
        collection_name=mongodb_config.image_metadata_collection
    )

    for image in tqdm(images, desc="Processing images"):
        logger.info(f"Processing {image}")

        localizations_data = localizer.localize(image, context=image_store_manager)

        localization_hashes = []
        for localization_data in localizations_data:
            localization = Localization(
                parent_image_hash=image.hash,
                model=str(localizer.model),
                **localization_data
            )

            db.upsert(db.config.localization_collection, localization)
            localization_hashes.append(localization.hash)

        # Update image with localization hashes and remove old localizations
        query = {
            "localization_hashes": {
                "$in": image.localization_hashes
            }
        }
        db.delete_many(db.config.localization_collection, query)
        image.localization_hashes = localization_hashes
        db.upsert(db.config.image_metadata_collection, image)